In [2]:
import pandas as pd
import io
import requests
import datetime
import plotly.express as px

## Get the Timeseries Data

Add documentation about the time series here.

In [3]:
from functions import get_time_series

confirmed, deaths, recovered, time_series_dates = get_time_series(local=True)

## Get all the Daily Reports 

Add documentation about these reports here.

In [4]:
from functions import get_daily_reports

daily_report_data, daily_report_dates = get_daily_reports(local=True)

Getting daily reports using local data
Failed to load data/03-14-2020.csv


## All Done Gathering Data.

Let's merge the time_series_data into one dataframe

In [ ]:
from functions import get_date_list
date_list = get_date_list(daily_report_dates)
print(confirmed.columns)
print(daily_report_data['03-12-2020'].columns)

In [ ]:
from math import log
today = daily_report_data['03-12-2020']
fig = px.scatter_geo(
    today,
    lat=today.Latitude,
    lon=today.Longitude,
    size=today.Confirmed
    # size of markers, "pop" is one of the columns of gapminder
)
fig.show()

In [ ]:
countries = sorted(confirmed['Country/Region'].drop_duplicates())
country_labels = [{'label': country, 'value': country} for country in countries]
print(country_labels)

In [5]:
us_confirmed = confirmed[(confirmed['Country/Region'] == 'US')]
county_confirmed = us_confirmed[us_confirmed['Province/State'].str.contains(',')]

us_deaths = deaths[(deaths['Country/Region'] == 'US')]
county_deaths = us_deaths[us_deaths['Province/State'].str.contains(',')]

us_recovered = recovered[(recovered['Country/Region'] == 'US')]
county_deaths = us_recovered[us_recovered['Province/State'].str.contains(',')]

from functions import list_of_states

state_confirmed = us_confirmed[us_confirmed['Province/State'].isin(list_of_states)]
state_confirmed['Province/State'].unique()

array(['Washington', 'New York', 'California', 'Massachusetts',
       'Diamond Princess', 'Grand Princess', 'Georgia', 'Colorado',
       'Florida', 'New Jersey', 'Oregon', 'Texas', 'Illinois',
       'Pennsylvania', 'Iowa', 'Maryland', 'North Carolina',
       'South Carolina', 'Tennessee', 'Virginia', 'Arizona', 'Indiana',
       'Kentucky', 'District of Columbia', 'Nevada', 'New Hampshire',
       'Minnesota', 'Nebraska', 'Ohio', 'Rhode Island', 'Wisconsin',
       'Connecticut', 'Hawaii', 'Oklahoma', 'Utah', 'Kansas', 'Louisiana',
       'Missouri', 'Vermont', 'Alaska', 'Arkansas', 'Delaware', 'Idaho',
       'Maine', 'Michigan', 'Mississippi', 'Montana', 'New Mexico',
       'North Dakota', 'South Dakota', 'West Virginia', 'Wyoming'],
      dtype=object)

In [ ]:
df = county_confirmed.copy()
new = df['Province/State'].str.split(',', 1, expand=True).rename(columns={0:'County',1:'State'})
df = pd.concat([df, new.reindex(df.index)], axis=1).copy()


In [18]:
from functions import us_state_abbrev
df = us_confirmed
new

,State
100,WA
101,NY
102,CA
103,MA
104,None
...,...
399,None
400,None
401,None
402,None


In [68]:
def assign_state(state_value):
    from functions import us_state_abbrev, abbrev_us_state
    if state_value in list(us_state_abbrev.keys()):
        # We have a State Name:
        state = state_value
        county = None
    elif ','in state_value:
        # We have a county, State Pair
        county, state_abbrev = [x.strip() for x in state_value.split(',')]
        state_abbrev = state_abbrev.replace('.','') # Watch out for D.C.!
        state = abbrev_us_state[state_abbrev]
    else:
        # It's a cruise ship!
        state = 'Cruise Ships'
        county = None
    return county, state


def parse_counties(df,col='Province/State'):
	df['State'] = df[col].apply(lambda x: us_state_abbrev.get(x, None)).copy()
	new2 = df[col].str.split(',', 1, expand=True).rename(columns={0:'County',1:'State'})
	df = pd.concat([df, new2.reindex(df.index)], axis=1).copy()
	return df

In [76]:
def get_states(df):
    df['location'] = pd.DataFrame(df['Province/State'].apply(lambda x: assign_state(x)))
    new_col_list = ['County','State']
    for n,col in enumerate(new_col_list):
        df[col] = df['location'].apply(lambda location: location[n])
    df = df.drop('location',axis=1)
    cols = df.columns.tolist()
    df = df[cols[-2:] + cols[:-2]].copy()
    return df

In [65]:
#us_confirmed['Province/State'=='Washington']['State'] = 'WA'
list(us_state_abbrev.keys())

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Northern Mariana Islands',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Palau',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virgin Islands',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']